In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vZz-lMKdy23_ZC6auvIcnWVCbZlA7q4w9-19mXz-rNF-',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'hvsmkproject-donotdelete-pr-41nlapna8gqstx'
object_key = 'Datasets.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
import keras
from keras.preprocessing.image import ImageDataGenerator

In [7]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [8]:
ls

Dataset/  forest1.h5  realtime.tgz


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
#Define the parameters/arguments for ImageDataGenerator class
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,rotation_range=180,zoom_range=0.2,horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [11]:
#Applying ImageDataGenerator functionality to trainset
x_train=train_datagen.flow_from_directory('Dataset/Dataset/train_set',target_size=(128,128),batch_size=32,class_mode='binary')

Found 436 images belonging to 2 classes.


In [12]:
#Applying ImageDataGenerator functionality to testset
x_test=test_datagen.flow_from_directory('Dataset/Dataset/test_set',target_size=(128,128),batch_size=32,class_mode='binary')

Found 121 images belonging to 2 classes.


In [13]:
x_train.class_indices

{'forest': 0, 'with fire': 1}

In [14]:
#import model building libraries

#To define Linear initialisation import Sequential
from keras.models import Sequential
#To add layers import Dense
from keras.layers import Dense
#To create Convolution kernel import Convolution2D
from keras.layers import Convolution2D
#import Maxpooling layer
from keras.layers import MaxPooling2D
#import flatten layer
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

In [15]:
#initializing the model
model=Sequential()

In [16]:
#add convolutional layer
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#add flatten layer 
model.add(Flatten())

In [17]:
#add hidden layer
model.add(Dense(150,activation='relu'))
#add output layer
model.add(Dense(1,activation='sigmoid'))

In [18]:
#configure the learning process
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])

In [19]:
len(x_train)

14

In [20]:
#Training the model
model.fit_generator(x_train,steps_per_epoch=14,epochs=10,validation_data=x_test,validation_steps=4)

Epoch 1/10
14/14 [==============================] - 30s 2s/step - loss: 4.2645 - accuracy: 0.5413 - val_loss: 0.2259 - val_accuracy: 0.9008
Epoch 2/10
14/14 [==============================] - 28s 2s/step - loss: 0.5275 - accuracy: 0.7523 - val_loss: 0.1959 - val_accuracy: 0.9008
Epoch 3/10
14/14 [==============================] - 28s 2s/step - loss: 0.2905 - accuracy: 0.8876 - val_loss: 0.1023 - val_accuracy: 0.9587
Epoch 4/10
14/14 [==============================] - 29s 2s/step - loss: 0.2094 - accuracy: 0.9151 - val_loss: 0.0828 - val_accuracy: 0.9752
Epoch 5/10
14/14 [==============================] - 28s 2s/step - loss: 0.1867 - accuracy: 0.9197 - val_loss: 0.0947 - val_accuracy: 0.9752
Epoch 6/10
14/14 [==============================] - 28s 2s/step - loss: 0.1858 - accuracy: 0.9243 - val_loss: 0.0901 - val_accuracy: 0.9752
Epoch 7/10
14/14 [==============================] - 28s 2s/step - loss: 0.1776 - accuracy: 0.9266 - val_loss: 0.0697 - val_accuracy: 0.9835
Epoch 8/10
14/14 [==

In [21]:
model.save("forest1.h5")

In [22]:
!tar  -zcvf realtime.tgz 'forest1.h5'

forest1.h5


In [23]:
!pip install ibm_watson_machine_learning

In [24]:
from ibm_watson_machine_learning import APIClient

In [25]:
wml_credentials={
    "url":"https://eu-de.ml.cloud.ibm.com",
    "apikey":"bzb9Z2uWa4hOsTkF_TvWHg7pHNOE_YRkasUP_iMGxeQa"
}

In [26]:
client = APIClient(wml_credentials)

In [27]:
client

In [28]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------  ------------------------
ID                                    NAME    CREATED
c996c044-2872-4fdf-bf98-e9183a0b5845  forest  2022-11-17T05:18:03.083Z
------------------------------------  ------  ------------------------


In [29]:
space_id="c996c044-2872-4fdf-bf98-e9183a0b5845"

In [30]:
client.set.default_space(space_id)

'SUCCESS'

In [31]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [32]:
software_space_uid = client.software_specifications.get_id_by_name("tensorflow_rt22.1-py3.9")

In [33]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [34]:
model_details = client.repository.store_model(model='realtime.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"cnn model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [35]:
model_id = client.repository.get_model_id(model_details)

In [36]:
model_id

'3ceb32be-1520-4590-8992-0fa70d5685af'

In [51]:
#Predictions
#import load_model from keras.model
from keras.models import load_model
#import image class from keras
from tensorflow.keras.preprocessing import image
#import numpy
import numpy as np
#import cv2
import cv2

In [3]:
#load the saved model
model = load_model("forest1.h5")

NameError: name 'load_model' is not defined

In [2]:
img=image.load_img('/content/drive/MyDrive/Dataset/test_set/with fire/180802_CarrFire_010_large_700x467.jpg')
x=image.img_to_array(img)
res = cv2.resize(x, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
#expand the image shape
x=np.expand_dims(res,axis=0)

NameError: name 'image' is not defined

In [31]:
pred=model.predict(x)

1/1 [==============================] - 0s 31ms/step


In [32]:
pred

array([[1.]], dtype=float32)